# Analysis Project on Data : Global Conflict Hashtag on Social Media
- **Linkedin :** Muhammad Aditya Bayhaqie
- **Email :** adityabayhaqie@gmail.com
- **Github :** bayhaqieee

## Business Questions

- When is the Highest Posting time and Lowest Posting time for the Most Hashtag?
- Which Conflict shows a High Exposure on Hashtag?
- What Content Summary for each Tag?
- Which Hashtag has the Most Comments, Likes and Views? (separated)
- Which Higher Engagement of Social Media Used for sharing the Hashtags?

## Data Preparation

#### Importing Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from datetime import datetime

#### Assigning Data

In [2]:
data = pd.read_csv("Data/conflicts_hashtag_search.csv")
data.head()

,fromSocial,text,likesCount,commentsCount,viewsCount,input,authorMeta/name,creationDate
0,youtube,NaN,NaN,NaN,NaN,yemencivilwar,NaN,NaN
1,youtube,Russia-Ukraine Conflict: Putin Warns NATO Risk...,22.0,7.0,3069.0,russiaukraineconflict,CNN-News18,2024-09-13T16:23:39.000Z
2,youtube,LIVE: Russia Launches Waves of Drone Attacks o...,118.0,21.0,16358.0,russiaukraineconflict,Firstpost,2024-10-01T00:17:49.000Z
3,youtube,Russian Forces Take Over Ukraine's Avdiivka | ...,166.0,56.0,18054.0,russiaukraineconflict,CNBC-TV18,2024-02-19T15:37:34.000Z
4,youtube,Russia-Ukraine War: Ukraine's Surprise Attack ...,53.0,4.0,6096.0,russiaukraineconflict,DD India,2024-08-24T13:36:51.000Z


**Insight:**
- There is in Total 1 Datasets Containing
    - fromSocial
        - Referred to which social media the posting being post
    - text
        - The Headline
    - likesCount
        - Numbers of Likes
    - commentsCount
        - Numbers of Comments
    - viewsCount
        - Numbers of Views
    - input
        - Hashtag (**PRIMARY**)
    - authorMeta/name
        - The Posters
    - creationDate
        - The Date of the Postings Created